In [1]:
import pathlib
import scanpy as sc
import scvi

/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
SAMPLE_RAW_DATA_PATH = DATA_DIR / 'raw_data' / 'GSM5226574_C51ctr_raw_counts.csv.gz'

In [3]:
sample_data = sc.read_csv(SAMPLE_RAW_DATA_PATH).T
sample_data

AnnData object with n_obs × n_vars = 6099 × 34546

## Doublet removal

In [4]:
# Filter genes expressed in less than 10 cells
sc.pp.filter_genes(sample_data, min_cells=10)

In [5]:
# Keep only top 2000 highly variable genes
sc.pp.highly_variable_genes(sample_data, n_top_genes=2000, subset=True, flavor='seurat_v3')

In [6]:
scvi.model.SCVI.setup_anndata(sample_data)
model = scvi.model.SCVI(sample_data)
model.train()

/home/victor/anaconda3/envs/rna/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [02:35<00:00,  2.72it/s, v_num=1, train_loss_step=317, train_loss_epoch=324]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [02:35<00:00,  2.57it/s, v_num=1, train_loss_step=317, train_loss_epoch=324]


In [7]:
solo = scvi.external.SOLO.from_scvi_model(model, sample_data)
solo.train()

INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 275/400:  69%|██████▉   | 275/400 [01:02<00:28,  4.39it/s, v_num=1, train_loss_step=0.192, train_loss_epoch=0.296]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.260. Signaling Trainer to stop.
